In [2]:
import cirq
import qiskit as q
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate, QFT
from qiskit.synthesis import SuzukiTrotter
from openfermion import QubitOperator
from qiskit.providers.aer.library import save_statevector


import numpy as np
import scipy, math, csv, os,sys
import matplotlib.pyplot as plt

In [2]:
def init_circ(n_x=2):
    # n_x = number of qubits in our register
    circ = q.QuantumCircuit(n_x, n_x)
    return circ

In [3]:
def create_ST_time_evo(Pauli_list, t, r):
    '''
    Construct a second-order Suzuki-Trotter time evolution decomposition
    
    Args:
    Pauli_list: list of summands in the given Hamiltonian
    t: time
    r: Trotter number
    
    Yield:
    evolution_list: A list containing the circuit gates in the order they should be implemented
    '''
    
    # Qiskit implementation
    hamil = SparsePauliOp.from_list(Pauli_list)
    evol_gate = PauliEvolutionGate(hamil, time=t, synthesis=SuzukiTrotter(reps=r))
    
    
    # Manual implementation
    evolution_list=[]
    numerical_factor = t/(2*r) #complex(0,-1)*t/(2*r)
    
    Pauli_list.reverse()
    for sublist in Pauli_list:
        summand = SparsePauliOp.from_list([sublist])
        sub_evol_gate = PauliEvolutionGate(summand,time=numerical_factor)
        evolution_list.append(sub_evol_gate)
    
    Pauli_list.reverse()
    for sublist in Pauli_list:
        summand = SparsePauliOp.from_list([sublist])
        sub_evol_gate = PauliEvolutionGate(summand,time=numerical_factor)
        evolution_list.append(sub_evol_gate)

    return evolution_list, evol_gate

In [4]:
def convert_op_string_to_usefulForm(op, n):
    '''
    Take a qubit operator operator, e.g. '.5 [Z0 Z1] + 4 [Z0 Z2]' and 
    construct it into a SparsePauliOp form to be used in time evolution
    
    args:
    op: The operator to be converted
    n: Total number of qubits
    
    Yield:
    Op_to_implement: The operator to implement in the circuit (can think of as ~'Hamiltonian')
    '''
    Pauli_list=[]
    
    for i in op:
        temp_str=''
        for j in range(n):
            temp_str+='I'
            
        term=str(i).split(" ")
        coeff = complex(term[0])
        
        term.pop(0)
        for op_term in term:
            if op_term !='[]':
                t_str=op_term.replace('[','')
                t_str=t_str.replace(']','')
                # t_list=t_str.split(" ")
                
                Pauli_op = t_str[0]
                qub_ind = int(t_str[1])
                temp_str = temp_str[::-1] # reverse the string so that python indexing matches qubit labelling
                
                if qub_ind==len(temp_str)-1:
                    temp_str = temp_str[:qub_ind]+Pauli_op
                    # print(qub_ind)
                elif qub_ind<len(temp_str)-1:
                    temp_str = temp_str[:qub_ind]+Pauli_op+temp_str[qub_ind+1:]
                temp_str = temp_str[::-1] # reverse back
        
        
        Pauli_list.append((temp_str,coeff))
            
        
    Op_to_implement=SparsePauliOp.from_list(Pauli_list)
    return Op_to_implement, Pauli_list
        
    

In [5]:
def calculate_error(Pauli_list,n, ind=0, start_r=501):
    '''
    Calculate the error arising from 2nd order Suzuki-Trotter decomposition
    
    Args:
    Pauli_list: A Pauli list of summands and numerical coeffs in the Hamiltonian being considered
    n: number of qubits (assuming homogenous bosonic modes)
    ind: Set to 0 in order to increment r by 1; set to 1 to increment r by 500
    
    Yield:
    Trotter_number: r value such that the error falls below the threshold 1e-6
    final_error: the exact error associated with r
    '''
    
    time_t=1
    
    Pauli_list.reverse() # IN order to be consistent with Qiskit implementation of 2nd order Suzuki-Trotter
    
    hamiltonian = SparsePauliOp.from_list(Pauli_list)
    ham_as_matrix = hamiltonian.to_matrix()
    # exact_evol_gate = PauliEvolutionGate(hamiltonian, time=time_t)
    
    # Step 1: Calculate exact U(t) value
    exact_ham_matrix = scipy.linalg.expm(complex(0,-1)*time_t*ham_as_matrix)
    
    if start_r>500:
        ST_reps=start_r-500
    else:
        ST_reps=start_r
        
    prev_error = 0
    err_counter=0
    # error=-1
    
    
    # APPROXIMATION 
    while ST_reps>0:
        # print("r = ", ST_reps)
        # approx_value=1
        
        # Step 2: Calculate approximated expectation value
        numerical_factor = complex(0,-1)*time_t/(2*ST_reps)
        omega = -1*time_t/(2*ST_reps)
        
        for index in range(len(Pauli_list)):
            sublist = Pauli_list[index]
        # for sublist in Pauli_list:
            summand = SparsePauliOp.from_list([sublist]) # The individual summand from the qubit operator expression
            summand_as_matrix = summand.to_matrix() 
            # sub_exp = np.add(np.cos(omega)*np.identity(pow(2,n)), complex(0,1)*np.sin(omega)*summand_as_matrix)
            summand_as_matrix *= numerical_factor
            sub_exp = scipy.linalg.expm(summand_as_matrix)
            
            if index==0:
                approx_value = sub_exp
            else:
                # approx_value = np.matmul(approx_value,sub_exp)
                approx_value = approx_value@sub_exp
                # approx_value = approx_value.dot(sub_exp)
    
        Pauli_list.reverse()
        for sublist in Pauli_list:
            summand = SparsePauliOp.from_list([sublist])
            summand_as_matrix = summand.to_matrix()
            # sub_exp = np.add(np.cos(omega)*np.identity(pow(2,n)), complex(0,1)*np.sin(omega)*summand_as_matrix)
            summand_as_matrix *= numerical_factor
            sub_exp = scipy.linalg.expm(summand_as_matrix)
            
            approx_value = approx_value@sub_exp
            # approx_value = approx_value.dot(sub_exp)

        
        final_approx_matrix_value=approx_value
        for i in range(ST_reps-1):
            final_approx_matrix_value = final_approx_matrix_value@approx_value
        
        # Step 3: Calculate error: epsilon = ||U(t)-\Tilde{U}(t)||
        error_matrix = np.subtract(exact_ham_matrix,final_approx_matrix_value)
        # error_matrix = exact_ham_matrix-final_approx_matrix_value
        error = np.linalg.norm(error_matrix)
        
        
        # Step 4: Repeat steps 2-3 with recursion over Trotter steps ST_reps until error is within certain bound
        if error<1e-6:
            break
        else:
            # print("error: ", error, "   error diff: ", (prev_error-error))
            if ind==0:
                ST_reps+=1
            elif ind==1:
                ST_reps+=500
            
    
        prev_error=error
        err_counter+=1
        
    Trotter_number = ST_reps
    final_error = error
    
    
    return Trotter_number, final_error
    

In [6]:
def create_circuit_v1(op_name, op_rep, d,ind,unary=0, r_step=0, start_r=501,refine=0):
    '''
    Given an operator and a list of d-levels, construct the circuit and perform 2nd-order 
    Suzuki-Trotter time evolution
    
    Args:
    op_name: The operator to be implemented, e.g. q2_0
    op_rep: Representation of the given operator (in charge encoding or Hermite)
    d: A list of d-levels
    ind: 0 if working in coordinate space; 1 in order to nest circuit with QFT and inv_QFT
    unary: 0 if non-unary encoding; 1 if unary encoding
    r_step: 0 if incrementing r by 1; 1 if incrementing by 500
    start_r: starting r value
    refine: 0 to build circuit; 1 to skip circuit construction
    
    Yield:
    qc: Quantum circuit with the operator implemented as gates
    r_error_list: list containing [Trotter number, error]
    '''
    
    # Step 1: Construct the base circuit
    tot_qub=0
    for register in d:
        if unary==0:
            num_q = int(math.log(register,2)) # number of qubits required for the register
            tot_qub+=num_q
        elif unary==1:
            tot_qub+=register
    
    qc=init_circ(tot_qub)
    # if unary==0:
    #     qc=init_circ(tot_qub)
    # elif unary==1:
    #     qc=init_circ(d[0])
    
    # Step 2: Convert the operator into a "Hamiltonian" that Qiskit will understand
    op = QubitOperator(op_rep)
    # print(op)
    hamiltonian = convert_op_string_to_usefulForm(op,tot_qub)[0]
    Pauli_list = convert_op_string_to_usefulForm(op,tot_qub)[1]
    
    # Step 3: Create time evolution operator
    r_error_list = calculate_error(Pauli_list, tot_qub, r_step, start_r)
    r = r_error_list[0]
    error = r_error_list[1]
    
    
    t=1 # time
    if r>0:
        evolution_info = create_ST_time_evo(Pauli_list, t, r)
        evolution_gate = evolution_info[1]
        evolution_list = evolution_info[0]
    elif r==0:
        evolution_info = create_ST_time_evo(Pauli_list, t, 1)
        evolution_gate = evolution_info[1]
        evolution_list = evolution_info[0]
    
    # Step 4: Apply time evolution ops to circuit
    q_list=[]
    for i in range(tot_qub):
        q_list.append(i)
    
    if refine==0:
        # print("Beginning circuit construction (r=", r,")")
        if ind==0:
            print()
            qc.append(evolution_gate, q_list)
            # for i in range(r):
            #     for sub_gate in evolution_list:
            #         qc.append(sub_gate,q_list)
        
        elif ind==1:
            print()
            # Peform QFT:
            qft = QFT(num_qubits=tot_qub)
            qc.append(qft,q_list)
        
            # Apply the Suzuki-Trotter decomposition
            qc.append(evolution_gate, q_list)
            # for i in range(r):
            #     for sub_gate in evolution_list:
            #         qc.append(sub_gate,q_list)
        
            # Perform inverse QFT:
            inv_qft = QFT(num_qubits=tot_qub, inverse=True)
            qc.append(inv_qft, q_list)
    
    return qc, r_error_list
    
    
    
    

In [7]:
def create_circuit(op_name,op_rep,d,ind,r,unary=0):
    '''
    Given an operator and a list of d-levels, construct the circuit and perform 2nd-order 
    Suzuki-Trotter time evolution
    
    Args:
    op_name: The operator to be implemented, e.g. q2_0
    op_rep: Representation of the given operator (in charge encoding or Hermite)
    d: A list of d-levels
    ind: 0 if working in coordinate space; 1 in order to nest circuit with QFT and inv_QFT
    r: Trotter number
    unary: 0 if non-unary encoding; 1 if unary encoding
    
    Yield:
    qc: Quantum circuit with the operator implemented as gates;
    Circuit_depth
    '''
    
    # Step 1: Construct the base circuit
    tot_qub=0
    for register in d:
        if unary==0:
            num_q = int(math.log(register,2)) # number of qubits required for the register
            tot_qub+=num_q
        elif unary==1:
            tot_qub+=register
    
    qc=init_circ(tot_qub)
    
    # Step 2: Convert the operator into a "Hamiltonian" that Qiskit will understand
    op = QubitOperator(op_rep)
    hamiltonian = convert_op_string_to_usefulForm(op,tot_qub)[0]
    Pauli_list = convert_op_string_to_usefulForm(op,tot_qub)[1]
    
    # Step 3: Create time evolution operator
    t=1
    Hamil_norm = np.linalg.norm(hamiltonian, ord='fro')
    H_t = 1/Hamil_norm
    evolution_gate = PauliEvolutionGate(hamiltonian, time=H_t*t, synthesis=SuzukiTrotter(reps=r))
    
    # Step 4: Apply time evolution ops to circuit
    q_list=[]
    for i in range(tot_qub):
        q_list.append(i)
    
    # print("Beginning circuit construction (r=", r,")")
    if ind==0:
        print()
        qc.append(evolution_gate, q_list)
        
    elif ind==1:
        print()
         # Peform QFT:
        qft = QFT(num_qubits=tot_qub)
        qc.append(qft,q_list)
        
        # Apply the Suzuki-Trotter decomposition
        qc.append(evolution_gate, q_list)
        
        # Perform inverse QFT:
        inv_qft = QFT(num_qubits=tot_qub, inverse=True)
        qc.append(inv_qft, q_list)

    
    return qc
    
    
    
    

In [8]:
def exact_unitary_matrix(op_rep,d,unary=0):
    '''
    Calculate the exact unitary matrix for time-evolution
    
    Args:
    op_rep: operator representation in given encoding
    d: list of d-levels
    
    Yield:
    ham_as_matrix: the matrix representation of the operator
    '''
    
    tot_qub=0
    for register in d:
        if unary==0:
            num_q = int(math.log(register,2)) # number of qubits required for the register
            tot_qub+=num_q
        elif unary==1:
            tot_qub+=register
            
    op = QubitOperator(op_rep)
    Pauli_list = convert_op_string_to_usefulForm(op,tot_qub)[1]
    
    Pauli_list.reverse() # IN order to be consistent with Qiskit implementation of 2nd order Suzuki-Trotter
    
    hamiltonian = SparsePauliOp.from_list(Pauli_list)
    ham_as_matrix = hamiltonian.to_matrix()
    
    return ham_as_matrix
    

In [ ]:
def Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,r=1,r_step=0,up_down=-1, upper_bound=0):
    '''
    Recursion over Trotter number r
    
    Args:
    r: Starting Trotter number
    r_step: the difference between previous and current Trotter number
    up_down: -1 if initiating algorithm; 0 if next Trotter step is less than current; 1 if next Trotter step is
            greater than current
    '''
     # Determining whether the op term is pure X, pure P, XP, or PX
    x_str=""
    p_str=""
    xp_str=""
    px_str=""
        
    counter0=0
    for c1_ind in range(len(op_name)):
        c=op_name[c1_ind].upper()
        if c=="X" or c=="Q":
            counter0+=1
            counter1=0
            for c2_ind in range(len(op_name)):
                E=op_name[c2_ind].upper()
                if E=="P" and c2_ind>c1_ind:
                    xp_str=op_name
                    counter1+=1
                elif E=="P" and c2_ind<c1_ind:
                    px_str=op_name
                    counter1+=1
            if counter1==0:
                x_str=op_name
                    
    if counter0==0:
        p_str=op_name
        
    
    print('r=',r)
    if True:
        if True:
            if x_str:
                circ = cdh.create_circuit(op_name,qub_operator,d_val,0,r,unary_ind)
            elif p_str:
                circ = cdh.create_circuit(op_name,qub_operator,d_val,1,r,unary_ind)
            elif xp_str:
                # First do p:
                circ=cdh.create_circuit(op_name,qub_operator[1],d_val,1,r,unary_ind)
                # Next do x:
                x_info=cdh.create_circuit(op_name,qub_operator[0],d_val,0,r,unary_ind)
                circ.append(x_info)
            elif px_str:
                # First do x:
                circ=cdh.create_circuit(op_name,qub_operator[0],d_val,0,r,unary_ind)
                # Next do p:
                p_info=cdh.create_circuit(op_name,qub_operator[1],d_val,1,r,unary_ind)
                circ.append(p_info)
    
    
    '''
    Fidelity calculation:
                    
    1. Obtain properly distributed states
    2. Calculate exact time-evolved state
    3. Calculate approximate time-evolved state; This is done by:
        First initialising a circuit in the random state
        Apply the circuit calculated above
        Retrieve state vector afterwards
    4. Calculate inner-product and subsequent fidelity
    '''
                 
    ### 1. Obtain properly distributed states ###
                    
    # number of qubits required for the register
    tot_qub=0
    for register in d_val:
        if unary_ind==0:
            num_q = int(math.log(register,2)) 
            tot_qub+=num_q
        elif unary_ind==1:
            tot_qub+=register
                    
    # Exact time evolution unitary matrix
    H = cdh.exact_unitary_matrix(qub_operator,d_val,unary_ind)
    H_norm = np.linalg.norm(U, ord='fro')
    H_t = 1/U_norm
    # U = np.array(U_t*U)
                    
                    
    ### Run an arbitrary number of iterations; take the mean, std. dev
    fidelity_list=[]
    iterations = int(1)
    M = pow(2,tot_qub)
            
    ''' Start iterations '''
    print(f"Beginning {iterations} iterations for time-evolved states.")
    for num_its in range(iterations):
        psi = np.array([])
                        
        # Constructing a random state
        for vect_val in range(M):
            a = np.random.normal()
            b = np.random.normal()
            sub_arr = np.array([a+1j*b])
            psi = np.append(psi,sub_arr)
                        
        # Normalise the state
        psi_norm = np.linalg.norm(psi)
        psi = (1/psi_norm) * psi
        # assert np.linalg.norm(psi)==1
        assert np.abs(np.linalg.norm(psi) - 1.0) < 1e-14 # Can get away with this threshold?
                    
                    
        ### 2. Calculate exact time-evolved state ###
        phi = scipy.sparse.linalg.expm_multiply(complex(0,1)*H_t*H,psi)
        phi_norm = np.linalg.norm(phi)
        phi = (1/phi_norm) * phi 
        assert np.abs(np.linalg.norm(phi) - 1.0) < 1e-14
                        
                        

        ### 3. Calculate approximated state ###
                        
        # Initialise circuit in state psi; append the circuit containing the 
        # second-order Suzuki-Trotter decomp. of the qubit operator
        backend = q.Aer.get_backend('statevector_simulator')
        qc = q.QuantumCircuit(tot_qub, tot_qub)
        qc.initialize(psi,[i for i in range(tot_qub)])
        qc.append(circ, [i for i in range(tot_qub)], [i for i in range(tot_qub)])
                        
        # Run the job and retrieve the statevector
        job = q.execute(qc, backend=backend,memory=True)
        qc.save_statevector()
        job_result = job.result()
        phi_tilde = job_result.get_statevector(qc)
        phi_tilde_norm = np.linalg.norm(phi_tilde)
        phi_tilde = (1/phi_tilde_norm) * phi_tilde
        assert np.abs(np.linalg.norm(phi_tilde) - 1)<5e-14
                        
                        
                        
        ### 4. Calculate inner product and fidelity ###
        inner = np.vdot(phi,phi_tilde) 
        inner_sq = inner * np.conj(inner)
        fidelity = 1-inner_sq
        # print(inner_sq,fidelity)
        assert np.imag(fidelity)==0
        assert np.real(fidelity)>=0
        assert np.real(fidelity)<=1
        fidelity_list.append(np.real(fidelity))
                
    ''' End iterations '''
                        
    # Calculataing statistics            
    mean = np.mean(fidelity_list)
    std_dev = np.std(fidelity_list)
    uncertainty = std_dev/iterations
    
    
    if mean <= 1e-6:
        if np.floor(r_step)==1 or np.floor(r_step)==0:
            Circuit_depth = circ.decompose().decompose().decompose().depth()
            return [r,Circuit_depth, mean, std_dev, uncertainty]
        elif upper_bound==0: 
            ''' 
            Reached the threshold for the first time
            We've found the upper bound; need to reduce r
            '''
            new_r = np.floor(r-(r_step/2))
            Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,new_r,r_step/2,0,1)
        else:
            new_r = np.floor(r-(r_step/2))
            Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,new_r,r_step/2,0,1)
            
            
    else:
        if upper_bound==0:
            '''
            Have not reached threshold at all yet
            Need to keep increasing r by 5000
            '''
            Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,r+5000,5000,1,0)
        elif upper_bound==1:
            '''
            Reached threshold on a previous r, so we reduced r, but failed to meet the threshold again
            Need to increae r by smaller amount
            '''
            new_r = np.floor(r+(r_step/2))
            Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,new_r,r_step/2,1,1)
        
    
    return


In [ ]:
def Hermite_Recursive_Trotter_scheme(op_name,qub_operator,d_val,unary_ind,r=1):
    '''
    Recursion over Trotter number r specifically for Hermite encoding
    Update: I thought that Hermite would require far fewer iterations than charge encoding, but that may
    not be the case, in which case this function will not be used.
    
    Args:
    r: Starting Trotter number
    '''
     # Determining whether the op term is pure X, pure P, XP, or PX
    x_str=""
    p_str=""
    xp_str=""
    px_str=""
        
    counter0=0
    for c1_ind in range(len(op_name)):
        c=op_name[c1_ind].upper()
        if c=="X" or c=="Q":
            counter0+=1
            counter1=0
            for c2_ind in range(len(op_name)):
                E=op_name[c2_ind].upper()
                if E=="P" and c2_ind>c1_ind:
                    xp_str=op_name
                    counter1+=1
                elif E=="P" and c2_ind<c1_ind:
                    px_str=op_name
                    counter1+=1
            if counter1==0:
                x_str=op_name
                    
    if counter0==0:
        p_str=op_name
        
    
    
    while r>0:
        if True:
            if x_str:
                circ = cdh.create_circuit(op_name,qub_operator,d_val,0,r,unary_ind)
            elif p_str:
                circ = cdh.create_circuit(op_name,qub_operator,d_val,1,r,unary_ind)
            elif xp_str:
                # First do p:
                circ=cdh.create_circuit(op_name,qub_operator[1],d_val,1,r,unary_ind)
                # Next do x:
                x_info=cdh.create_circuit(op_name,qub_operator[0],d_val,0,r,unary_ind)
                circ.append(x_info)
            elif px_str:
                # First do x:
                circ=cdh.create_circuit(op_name,qub_operator[0],d_val,0,r,unary_ind)
                # Next do p:
                p_info=cdh.create_circuit(op_name,qub_operator[1],d_val,1,r,unary_ind)
                circ.append(p_info)
    
    
        '''
        Fidelity calculation:
                    
        1. Obtain properly distributed states
        2. Calculate exact time-evolved state
        3. Calculate approximate time-evolved state; This is done by:
            First initialising a circuit in the random state
            Apply the circuit calculated above
            Retrieve state vector afterwards
        4. Calculate inner-product and subsequent fidelity
        '''
                 
        ### 1. Obtain properly distributed states ###
                    
        # number of qubits required for the register
        tot_qub=0
        for register in d_val:
            if unary_ind==0:
                num_q = int(math.log(register,2)) 
                tot_qub+=num_q
            elif unary_ind==1:
                tot_qub+=register
                    
        # Exact time evolution unitary matrix
        H = cdh.exact_unitary_matrix(qub_operator,d_val,unary_ind)
        H_norm = np.linalg.norm(U, ord='fro')
        H_t = 1/U_norm
        # U = np.array(U_t*U)
                    
                    
        ### Run an arbitrary number of iterations; take the mean, std. dev
        fidelity_list=[]
        iterations = int(50)
        M = pow(2,tot_qub)
            
        ''' Start iterations '''
        # print(f"Beginning {iterations} iterations for time-evolved states.")
        for num_its in range(iterations):
            psi = np.array([])
                        
            # Constructing a random state
            for vect_val in range(M):
                a = np.random.normal()
                b = np.random.normal()
                sub_arr = np.array([a+1j*b])
                psi = np.append(psi,sub_arr)
                        
            # Normalise the state
            psi_norm = np.linalg.norm(psi)
            psi = (1/psi_norm) * psi
            # assert np.linalg.norm(psi)==1
            assert np.abs(np.linalg.norm(psi) - 1.0) < 1e-14 # Can get away with this threshold?
                    
                    
            ### 2. Calculate exact time-evolved state ###
            phi = scipy.sparse.linalg.expm_multiply(complex(0,-1)*H_t*H,psi)
            phi_norm = np.linalg.norm(phi)
            phi = (1/phi_norm) * phi 
            assert np.abs(np.linalg.norm(phi) - 1.0) < 1e-14
                        
                        

            ### 3. Calculate approximated state ###
                        
            # Initialise circuit in state psi; append the circuit containing the 
            # second-order Suzuki-Trotter decomp. of the qubit operator
            backend = q.Aer.get_backend('statevector_simulator')
            qc = q.QuantumCircuit(tot_qub, tot_qub)
            qc.initialize(psi,[i for i in range(tot_qub)])
            qc.append(circ, [i for i in range(tot_qub)], [i for i in range(tot_qub)])
                        
            # Run the job and retrieve the statevector
            job = q.execute(qc, backend=backend,memory=True)
            qc.save_statevector()
            job_result = job.result()
            phi_tilde = job_result.get_statevector(qc)
            phi_tilde_norm = np.linalg.norm(phi_tilde)
            phi_tilde = (1/phi_tilde_norm) * phi_tilde
            assert np.abs(np.linalg.norm(phi_tilde) - 1)<5e-14
                        
                        
                        
            ### 4. Calculate inner product and fidelity ###
            inner = np.vdot(phi,phi_tilde) 
            inner_sq = inner * np.conj(inner)
            fidelity = 1-inner_sq
            # print(inner_sq,fidelity)
            assert np.imag(fidelity)==0
            assert np.real(fidelity)>=0
            assert np.real(fidelity)<=1
            fidelity_list.append(np.real(fidelity))
                
        ''' End iterations '''
                        
        # Calculataing statistics            
        mean = np.mean(fidelity_list)
        std_dev = np.std(fidelity_list)
        uncertainty = std_dev/iterations
        
        if mean<=1e-6:
            break
        else:
            r+=1
    
    
    Circuit_depth = circ.decompose().decompose().decompose().depth()
    
    return [r,Circuit_depth, mean, std_dev, uncertainty]
    

In [14]:
def calc_inner_prod_sq(psi,phi):
    '''
    Calculate |<psi|phi>|^2
    '''
    inner = np.vdot(psi,phi)
    inner_sq = inner * np.conj(inner)
    
    return inner_sq

In [ ]:
def get_state_vector_from_circ(tot_qub, psi, circ):
    '''
    Initialise a given circuit in the state psi, then retrieve the new statevector
    '''
    
    # Initialise circuit in state psi; append the circuit containing the 
    # second-order Suzuki-Trotter decomp. of the qubit operator
    backend = q.Aer.get_backend('statevector_simulator')
    qc = q.QuantumCircuit(tot_qub, tot_qub)
    qc.initialize(psi,[i for i in range(tot_qub)])
    qc.append(circ, [i for i in range(tot_qub)], [i for i in range(tot_qub)])
    
    # Execute the circuit and retrieve the statevector
    job = q.execute(qc, backend=backend,memory=True)
    qc.save_statevector()
    job_result = job.result()
    phi_tilde = job_result.get_statevector(qc)
    
    return phi_tilde

In [3]:
def fidelity(circ, H, t, psi, tot_qub):
    '''
    Calculate state fidelity
    '''
    
    # Normalise the state
    psi_norm = np.linalg.norm(psi)
    psi = (1/psi_norm) * psi
    # assert np.linalg.norm(psi)==1
    if np.abs(np.linalg.norm(psi) - 1.0) > 2e-14:
    # if np.abs(np.linalg.norm(psi) - 1.0) != 0:
        print("ERROR: Initial state psi is not normalised")
        assert np.abs(np.linalg.norm(psi) - 1.0) <= 2e-14 
        return
    
    
    ### 2. Calculate exact time-evolved state ###
    phi = scipy.sparse.linalg.expm_multiply(complex(0,-1)*t*H,psi)
    if np.abs(np.linalg.norm(phi) - 1.0) > 2e-14:
        print("ERROR: Exact time-evolved state phi is not normalised")
        assert np.abs(np.linalg.norm(phi) - 1.0) <= 2e-14
        return
        
        
    
    ### 3. Calculate approximated state ###
    phi_tilde = get_state_vector_from_circ(tot_qub, psi, circ)
    
    if np.abs(np.linalg.norm(phi_tilde) - 1)>5e-14:
        print("ERROR: Approximated state phi_tilde is not normalised; 1 - norm = ", np.abs(np.linalg.norm(phi_tilde) - 1))
        assert np.abs(np.linalg.norm(phi_tilde) - 1)<=5e-14 # will fail and halt the computation
        return
    
    
    ### 4. Calculate inner product and fidelity ###
    inner_sq = calc_inner_prod_sq(phi, phi_tilde)
    
    if np.real(inner_sq)>1:
        if np.abs(1-np.real(inner_sq))>5e-14:
            print(f"ERROR: Inner product significantly greater than 1: {inner_sq}, delta = {np.abs(1-np.real(inner_sq))}")
            assert np.abs(1-np.real(inner_sq))<=5e-14 # will fail and halt the computation
            return
        else:
            inner_sq=1
        
        
    fidelity = 1-inner_sq
    # print(inner_sq,fidelity)
    
    if np.imag(fidelity)!=0:
        print("ERROR: Fidelity has imaginary component")
        assert np.imag(fidelity)==0 # will fail and halt the computation
        return
        
    if np.real(fidelity)>1:
        if np.abs(1-np.real(fidelity))>5e-14:
            print("ERROR: Fidelity is significantly larger than 1")
            assert np.real(fidelity)<=1 # will fail and halt the computation
            return
        else:
            fidelity=1
        
    if np.real(fidelity)<0:
        if np.abs(np.real(fidelity))>5e-14:
            print("ERROR: Fidelity is negative and has large magnitude")
            assert np.real(fidelity)>=0 # will fail and halt the computation
            return       
        else:
            fidelity=0
    
    if np.abs(np.real(fidelity))<5e-14:
        fidelity = 0
    
    return fidelity